<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Paola Silva Muñoz
- Nombre de alumno 2: Carla Guzmán Roa



### **Link de repositorio de GitHub:** `https://github.com/Paito249/MDS7202`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Universidad/Lab_datos/Lab_07'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [4]:
df_retail = pd.read_pickle("/content/drive/MyDrive/Universidad/Lab_datos/Lab_7/online_retail_II_cleaned.pickle")
#df_retail = pd.read_pickle(f'{path}/online_retail_II_cleaned.pickle')
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [5]:
def custom_features(df_in):
    #Por conveniencia para el desarrollo de todas las secciones, se crea una nueva columna al df del total gastado
    #en cada compra.
    df_in['Total'] = df_in['Price']*df_in['Quantity']

    #además ordenaremos el dataframe según 'InvoiceDate' para usarlo en orden en el cálculo de Periodicity
    df_in = df_in.sort_values(by=['Customer ID', 'InvoiceDate'])

    #se crea un objeto "GroupBy" los datos por el Customer ID para poder aplicar funciones de agregación por grupo.
    agrupados = df_in.groupby('Customer ID')

    #Se crea el dataframe de resultado
    LRMFP = pd.DataFrame()

    #df para guardar valores auxiliares
    L = pd.DataFrame()

    #Para calcular Length se obtiene el valor de L para cada cliente según el Customer ID, y se guarda en L.
    L['First Visit'] = agrupados['InvoiceDate'].min()
    L['Last Visit'] = agrupados['InvoiceDate'].max()
    #Se calcula la diferencia entre la primera y la última visita
    LRMFP['Length'] = L['Last Visit'].dt.date - L['First Visit'].dt.date
    #se transforma a entero la cantidad de días
    LRMFP['Length'] = LRMFP['Length'].dt.days

    #Para Recency primero fijamos el valor del "día actual" (último día en el df + 1)
    dia_max = df_in['InvoiceDate'].dt.date.max() + datetime.timedelta(days=1)
    #Luego con el valor máximo del "GroupBy" y se hace la resta correspondiente.
    LRMFP['Recency'] = dia_max - L['Last Visit'].dt.date
    #se transforma a entero la cantidad de días
    LRMFP['Recency'] = LRMFP['Recency'].dt.days

    #Para Monetary, se aplica el promedio del total al GroupBy
    LRMFP['Monetary'] = agrupados['Total'].mean()

    #Para Frecuency simplemente se cuentan la cantidad de visitas del cliente en el objeto agrupado
    LRMFP['Frequency'] = agrupados.size()

    #Para Periodicity se calcula la diferencia entre elementos consecutivos para cada grupo. Además, se
    #convierte a días el resultado.
    df_in['IVT'] = agrupados['InvoiceDate'].diff().dt.total_seconds()/ (60 * 60 * 24)

    #luego calculamos la desviación estándar de los valores calculados por grupo
    LRMFP['Periodicity'] = agrupados['IVT'].std()
    LRMFP['Periodicity']= LRMFP['Periodicity'].fillna(0)


    return LRMFP

In [6]:
lrmfp_df=custom_features(df_retail)
lrmfp_df

,Length,Recency,Monetary,Frequency,Periodicity
Customer ID,,,,,
12346.0,196,165,11.298788,33,21.789202
12347.0,37,3,18.638310,71,4.425417
12348.0,0,74,11.108000,20,0.000000
12349.0,182,43,26.187647,102,16.302199
12351.0,0,11,14.330000,21,0.000000
...,...,...,...,...,...
18283.0,276,18,2.854240,217,11.854516
18284.0,0,67,16.488571,28,0.000000
18285.0,0,296,35.583333,12,0.000000


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
class MinMax(BaseEstimator, TransformerMixin):


    #Se hereda fit desde TransformerMixin
    def fit(self, X, y=None):
        self.min_val = X.min()
        self.max_val = X.max()
        return self

    #Se aplica la transformación de la fórmula del enunciado
    def transform(self, X):
        X_t = (X - self.min_val) / (self.max_val - self.min_val)
        return X_t

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [8]:
pipeline =  Pipeline([
    ('caractEXtract',FunctionTransformer(custom_features)),
     ('prepros',ColumnTransformer([('minMax',MinMax(),['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity'])])),
     ('tsne',TSNE(n_components = 2,random_state=42 ))
])

In [9]:
reduccion_tsne=pipeline.fit_transform(df_retail)
tsne_df= pd.DataFrame(reduccion_tsne,columns=["0","1"])
fig = px.scatter(tsne_df , x="0",y="1",title="Componentes relevantes t-sne df retail")
fig

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [10]:
inercias_array=[]
for k in range(1,21):
  pipeline_kmeans =  Pipeline([
        ('caractEXtract',FunctionTransformer(custom_features) ),
        ('prepros',ColumnTransformer([('minMax',MinMax(),['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity'])])),
        ('kmeans',KMeans(n_clusters = k,init='k-means++',random_state=42,n_init=10))
    ])
  pipeline_kmeans.fit(df_retail)
  inercias_array.append(pipeline_kmeans['kmeans'].inertia_)

df_inercias = pd.DataFrame({'n_cluster':range(1,21),
                         'inercia':inercias_array
                         })

In [11]:
pipeline_kmeans['prepros'].feature_names_in_

array(['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity'],
      dtype=object)

In [12]:
fig=px.line(df_inercias, x='n_cluster', y='inercia', title='Codo K-means')
fig

De acuerdo al gráfico de inercias es posible observar que la curva tiene una disminución significativa hasta el cluster 3, de ahí la disminución no es significativa y podría llevar a la perdida de la generalización. Por lo que, para este caso se eligirá n_cluster 3.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [13]:
pipeline_kmeans =  Pipeline([
      ('caractEXtract',FunctionTransformer(custom_features) ),
      ('prepros',ColumnTransformer([('minMax',MinMax(),['Length', 'Recency',
                                      'Monetary', 'Frequency', 'Periodicity'])])),
      ('kmeans',KMeans(n_clusters = 3,init='k-means++',random_state=42,n_init=10))
  ])
pipeline_kmeans.fit(df_retail)
kmeans=pipeline_kmeans['kmeans']
comportamiento=lrmfp_df.copy()
comportamiento['cluster']=kmeans.labels_
comportamiento.groupby('cluster').mean(numeric_only=True)

,Length,Recency,Monetary,Frequency,Periodicity
cluster,,,,,
0,277.911086,37.311199,32.380861,167.146877,21.174747
1,23.786236,251.630865,53.258432,28.454640,3.631892
2,39.637516,54.576679,32.285199,48.567174,5.252906


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

Es posible observar una segmentación clara de clientes, donde cada cluster representa el perfil del tipo de cliente.

Se puede inferir que el **cluster 0** se refiere a clientes que han realizado compras recientemente, debido a que poseen una Recency relativamente baja. Además, son clientes habituales, ya que tienen una Frequency alta, pero la Periodicity es moderada, lo que sugiere que no compran con tanta regularidad. En cuanto al Length, se puede notar que han sido clientes por un largo período de tiempo. Por último, el valor de Monetary puede sugerir que gastan una cantidad elevada en sus compras.

Al analizar el **cluster 1**, podemos observar que son clientes con una Recency alta, lo que indicaría que ha pasado un largo período de tiempo desde su última compra. Además, son clientes con Frequency y Periodicity bajos, por lo que no son compradores habituales. En lo que respecta al Length, podemos observar que son clientes que poseen una historia muy larga siendo clientes de la tienda. Finalmente, Monetary refleja que son clientes que gastan valores significativos en sus compras a pesar de no ser habituales.


En cuanto al **cluster 2**, se puede observar que poseen valores moderados en todos los ámbitos. Son clientes habituales y compran de manera regular debido a que tienen una alta Frequency, pero no necesariamente constante debido a su Periodicity. Además, no han interactuado con la empresa durante mucho tiempo, ya que su Length es moderada en comparación con los anteriores, y poseen una Recency y Monetary también moderados.

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [14]:
fig = px.scatter(tsne_df, x="0",y="1",color=kmeans.labels_,
                 title="Componentes relevantes t-sne")
fig

Se puede decir que en general los clústers están bien separados, a pesar de que tienen una pequeña superposición. El clúster rosado tiene la distinción más clara de de los 3, ya que la superposición de puntos es muy pequeña con los otros 2 clústers. Por otor lado, entre el clúster amarillo y el azul igual se puede observar una definición clara, pero el área de superposición de punto es un poco mayor que en el primer caso explicado. De cualquier forma, esta superposición no es crítica y los clúster se separan claramente en el espacio 2-D.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>